In [21]:
# Imports
from collections import OrderedDict
import json
import pprint

## Assignments

### Input Assignments

In [1]:
# Json input
input_str= """
{
    "I1": [
        "boolean",
        "false"
    ],
    "I2": [
        ["1", "2", "3"],
        "1"
    ],
    "STATUS": [
        ["stopped", "running"],
        "stopped"
    ]
    
}
"""

In [2]:
# Expected NuSMV
expected= 'init(I1):= false;\n' + \
    'next(I1):= case\n' + \
    '   stab: {true, false};\n' +\
    '   true: I1;\n' + \
    'esac;\n' + \
    'init(I1):= "1";\n' + \
    'next(I2):= case\n' + \
    '   stab: {"1", "2", "3"};\n' +\
    '   true: I2;\n' + \
    'esac;\n'+ \
    'init(I1):= "stopped";\n' + \
    'next(I3):= case\n' + \
    '   stab: {"stopped", "running"};\n' +\
    '   true: I3;\n' + \
    'esac;\n'

In [7]:
# Convert Json string to python dictionary
input_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [8]:
input_dict

OrderedDict([('I1', ['boolean', 'false']),
             ('I2', [['1', '2', '3'], '1']),
             ('STATUS', [['stopped', 'running'], 'stopped'])])

In [37]:
# define initialization & assignment strings
init_str= ""
assign_str= ""

In [38]:
# loop through elements
for key,value in input_dict.items():
    # Check if type is a string
    if isinstance(value[0], str):
        # Initialization
        init_str= init_str + "init({0}):= ".format(key) + \
            input_dict[key][1] + ";\n"

        # Assignment
        assign_str= assign_str + \
            'next({0}):= case\n'.format(key) + \
            '   stab: {true, false};\n' +\
            '   true: {0};\n'.format(key) + \
            'esac;\n'
        
    # if type is not string
    else:
        # Initialization
        # Add quotes to initial value for non-booleans
        init_str= init_str + "init({0}):= ".format(key) + \
            '"{0}"'.format(input_dict[key][1]) + ";\n"

        assign_type= "{"
        for idx, item in enumerate(value[0]):
            # If not last item
            if idx < len(value[0])-1:
                assign_type= assign_type + '"' + item + '"' + ", "
            # If last item
            else:
                assign_type= assign_type + '"' + item + '"'
        
        assign_type= assign_type + "}"

        # Assignment
        assign_str= assign_str + \
            'next({0}):= case\n'.format(key) + \
            '   stab: {0};\n'.format(assign_type) +\
            '   true: {0};\n'.format(key) + \
            'esac;\n'

In [39]:
out_str= init_str + assign_str

In [40]:
out_str

'init(I1):= false;\ninit(I2):= "1";\ninit(STATUS):= "stopped";\nnext(I1):= case\n   stab: {true, false};\n   true: I1;\nesac;\nnext(I2):= case\n   stab: {"1", "2", "3"};\n   true: I2;\nesac;\nnext(STATUS):= case\n   stab: {"stopped", "running"};\n   true: STATUS;\nesac;\n'

In [41]:
pprint.pprint(out_str)

('init(I1):= false;\n'
 'init(I2):= "1";\n'
 'init(STATUS):= "stopped";\n'
 'next(I1):= case\n'
 '   stab: {true, false};\n'
 '   true: I1;\n'
 'esac;\n'
 'next(I2):= case\n'
 '   stab: {"1", "2", "3"};\n'
 '   true: I2;\n'
 'esac;\n'
 'next(STATUS):= case\n'
 '   stab: {"stopped", "running"};\n'
 '   true: STATUS;\n'
 'esac;\n')
